In [24]:
import pandas as pd

# Load the Excel file
file_path = r"C:\Users\HP\Downloads\panas score trial.xlsx"  # Use raw string
updated_sheet_data = pd.read_excel(file_path, sheet_name="Sheet7")  # Change "Sheet1" if needed

# Display the first few rows of the dataset
print(updated_sheet_data.head())

  timestamp from (2016-05-25 04:15:12) to (2016-06-02 22:43:42)  \
0                                         1460000000              
1                                         1460000000              
2                                         1460000000              
3                                         1460000000              
4                                         1460000000              

   Acceleration average- raw_acc:magnitude_stats:mean  \
0                                           1.101873    
1                                           1.109824    
2                                           1.025130    
3                                           1.015322    
4                                           1.012749    

   Maximum acceleration- raw_acc:magnitude_stats:percentile75  \
0                                           1.362276            
1                                           1.419649            
2                                           1.065818       

In [25]:
# Define weights for Positive Affect (PA) and Negative Affect (NA) calculation
weights_pa = {
    "Normalized Acceleration": 0.4,
    "Normalized Gyro": 0.3,
    "Average illuminance- lf_measurements:light & atmospheric press": 0.3,
}
weights_na = {
    "Acceleration variance- raw_acc:magnitude_stats:std (variance= std^2)": 0.4,
    "proc_gyro:3d:std_x": 0.3,
    "Average illuminance- lf_measurements:light": -0.3,  # Low light contributes negatively
}

# Ensure no NaN values for calculation
updated_sheet_data.fillna(updated_sheet_data.mean(), inplace=True)

# Calculate PA and NA scores as weighted sums
updated_sheet_data["PA Score"] = (
    updated_sheet_data["Normalized Acceleration"] * weights_pa["Normalized Acceleration"]
    + updated_sheet_data["Normalized Gyro"] * weights_pa["Normalized Gyro"]
    + updated_sheet_data["Average illuminance- lf_measurements:light"] * weights_pa["Average illuminance- lf_measurements:light"]
)

updated_sheet_data["NA Score"] = (
    updated_sheet_data["Acceleration variance- raw_acc:magnitude_stats:std (variance= std^2)"] * weights_na["Acceleration variance- raw_acc:magnitude_stats:std (variance= std^2)"]
    + updated_sheet_data["proc_gyro:3d:std_x"] * weights_na["proc_gyro:3d:std_x"]
    + updated_sheet_data["Average illuminance- lf_measurements:light"] * weights_na["Average illuminance- lf_measurements:light"]
)

# Normalize PA and NA scores to the PANAS range (10-50)
updated_sheet_data["PA Score"] = 10 + (updated_sheet_data["PA Score"] - updated_sheet_data["PA Score"].min()) / (
    updated_sheet_data["PA Score"].max() - updated_sheet_data["PA Score"].min()
) * 40

updated_sheet_data["NA Score"] = 10 + (updated_sheet_data["NA Score"] - updated_sheet_data["NA Score"].min()) / (
    updated_sheet_data["NA Score"].max() - updated_sheet_data["NA Score"].min()
) * 40

# Display the updated dataset with the calculated PANAS scores
updated_sheet_data[["PA Score", "NA Score"]].head()


,PA Score,NA Score
0,10.073739,49.305384
1,10.140998,49.670170
2,10.196384,47.861359
3,19.553714,18.646407
4,19.552931,18.644364


In [26]:
output_path = r"C:\Users\HP\Downloads\panas_scores_final.xlsx"
updated_sheet_data.to_excel(output_path, index=False)
print(f"PANAS scores saved to {output_path}")

PANAS scores saved to C:\Users\HP\Downloads\panas_scores_final.xlsx
